In [1]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13372899743652974314
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3643801600
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6319837650755708353
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [2]:
pip install pandas yfinance requests beautifulsoup4 transformers langchain torch


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas requests beautifulsoup4 tqdm


Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import concurrent.futures
from tqdm import tqdm

# List of 100 ticker symbols
tickers = [
    'AAPL', 'MSFT', 'NVDA', 'GOOGL', 'AI', 'GOOG', 'AMZN', 'META', 'TSM', 'TSLA', 
    'PINS', 'IBM', 'PFE', 'VZ', 'PDD', 'DIS', 'NOW', 'BABA', 'BX', 'MS', 
    'AMAT', 'CAT', 'GS', 'NEE', 'ISRG', 'HSBC', 'CMCSA', 'RTX', 'RY', 'TTE', 
    'UL', 'SPGI', 'UNP', 'HDB', 'T', 'BHP', 'LOW', 'HON', 'LMT', 'VRTX', 
    'SNY', 'MUFG', 'TJX', 'BLK', 'UBER', 'ARM', 'SYK', 'COP', 'PGR', 'BUD', 
    'BKNG', 'INTC', 'ELV', 'C', 'ETN', 'REGN', 'PLD', 'SCHW', 'MU', 'UPS', 
    'NKE', 'BSX', 'MMC', 'CB', 'BA', 'RIO', 'ADI', 'LRCX', 'ADP', 'AMT', 
    'KKR', 'ANET', 'MDT', 'SONY', 'PANW', 'TD', 'KLAC', 'IBN', 'MMM', 'GILD', 
    'PEP', 'ABT', 'MRK', 'WMT', 'KO', 'CMG', 'MCD', 'JNJ', 'V', 'MA', 
    'PYPL', 'NFLX', 'ADBE', 'CRM', 'CSCO', 'ORCL', 'SAP', 'INTU', 'AMD', 'QCOM'
]

# URLs to fetch articles from
urls = [
    "https://www.reuters.com/technology/",
    "https://www.bloomberg.com/markets",
    "https://www.marketwatch.com/latest-news",
    "https://www.cnbc.com/world/?region=world",
    "https://www.wsj.com/news/markets",
    "https://www.ft.com/markets",
    "https://finance.yahoo.com/",
    "https://www.investing.com/news/stock-market-news",
    "https://seekingalpha.com/market-news",
    "https://www.fool.com/investing-news/",
    "https://www.nasdaq.com/news/market-news",
    "https://www.barrons.com/market-data",
    "https://www.forbes.com/investing/",
    "https://www.businessinsider.com/markets",
    "https://www.zacks.com/investment-news",
    "https://www.thestreet.com/markets",
    "https://www.morningstar.com/market-news",
    "https://www.benzinga.com/markets",
    "https://www.bloombergquint.com/markets",
    "https://www.investors.com/market-trend/stock-market-today/",
    "https://edition.cnn.com/business/markets",
    "https://www.msn.com/en-us/money/markets",
    "https://finance.yahoo.com/news/",
]

# Function to fetch articles from a single URL
def fetch_articles_from_url(url, ticker):
    headers = {'User-Agent': 'Mozilla/5.0'}
    articles = []
    try:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Error fetching articles from {url}: {response.status_code}")
            return articles

        soup = BeautifulSoup(response.content, 'html.parser')
        article_elements = soup.find_all('article')
        for article in article_elements[:10]:  # Limit to 10 articles per URL
            title = article.find('h3').get_text() if article.find('h3') else 'No title'
            description = article.find('p').get_text() if article.find('p') else 'No description'
            link = article.find('a')['href'] if article.find('a') else 'No link'
            published_at = article.find('time')['datetime'] if article.find('time') else 'No date'
            articles.append({
                'ticker': ticker,
                'title': title,
                'description': description,
                'url': link,
                'publishedAt': published_at
            })
    except Exception as e:
        print(f"Error fetching articles from {url} for {ticker}: {e}")
    return articles

# Use concurrent.futures to parallelize the fetching process
def load_articles_for_tickers(tickers, urls):
    all_articles = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(fetch_articles_from_url, url, ticker): (url, ticker) for ticker in tickers for url in urls}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            url, ticker = futures[future]
            try:
                articles = future.result()
                all_articles.extend(articles)
            except Exception as e:
                print(f"Error fetching articles for {ticker} from {url}: {e}")
    return all_articles

# Load articles
articles = load_articles_for_tickers(tickers, urls)

# Check if articles list is empty
if articles:
    # Save to CSV
    news_df = pd.DataFrame(articles)
    news_df.to_csv('news_articles_100_tickers.csv', index=False)
    print("News articles gathered and saved to 'news_articles_100_tickers.csv'.")
else:
    print("No news articles found for any ticker.")

# Optional: Display the first few rows of the DataFrame to verify
if articles:
    print(pd.DataFrame(articles).head())


  0%|                                                                                 | 2/2300 [00:00<05:58,  6.42it/s]

Error fetching articles from https://www.reuters.com/technology/: 401
Error fetching articles from https://www.marketwatch.com/latest-news: 401
Error fetching articles from https://www.wsj.com/news/markets: 401


  0%|▏                                                                                | 5/2300 [00:00<08:22,  4.57it/s]

Error fetching articles from https://www.investing.com/news/stock-market-news for AAPL: 'href'


  0%|▎                                                                                | 8/2300 [00:01<10:04,  3.79it/s]

...

Error fetching articles from https://www.thestreet.com/markets: 404
Error fetching articles from https://www.zacks.com/investment-news: 404
Error fetching articles from https://www.morningstar.com/market-news: 404


100%|█████████████████████████████████████████████████████████████████████████████▋| 2290/2300 [46:39<00:01,  5.36it/s]

Error fetching articles from https://www.investors.com/market-trend/stock-market-today/: 403


100%|██████████████████████████████████████████████████████████████████████████████| 2300/2300 [46:44<00:00,  1.22s/it]

News articles gathered and saved to 'news_articles_100_tickers.csv'.
  ticker                                              title  \
0   AAPL                                    News & Insights   
1   AAPL  8 out of 10 S&P 500 materials stocks deliver E...   
2   AAPL  Libya is said to begin cutting output at bigge...   
3   AAPL  19 out of 25 Information Technology companies ...   
4   AAPL  Boeing's new CEO will confront key issues with...   

                                         description  \
0  Options trading has become increasingly popula...   
1                                     No description   
2                                     No description   
3                                     No description   
4                                     No description   

                                                 url publishedAt  
0                       /market-activity/stocks/adbe     No date  
1  /news/4133384-8-out-of-10-sp-500-materials-sto...     No date  
2  /news/41336